# CNN 문장분류기
---

## 0. 와이드 스크린으로 만들기

In [27]:
# Wide display
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

---
## 1. 필요한 라이브러리를 불러온다.

***tensorflow***: 핵심 딥러닝 프레임워크

***numpy***: 수치 연산 핵심 프레임워크

***pandas***: 데이터를 dataframe이라는 형식에 담아 편리하게 관리하기 위한 라이브러리

***gensim***: Word2Vec 관련 라이브러리. 여기서는 pre-trained word vector를 불러오기 위해서 사용.

***nltk***: tokenize와 관련된 라이브러리

***pprint***: 출력 결과를 예쁘게 정렬 해줌.

In [28]:
import tensorflow as tf
import numpy as np
import pandas as pd
import gensim
from gensim.models.word2vec import Word2Vec # for Word2Vec and Topic modeling
from nltk.tokenize import word_tokenize, regexp_tokenize # for tokenizing
from nltk.corpus import stopwords # for stopwords filtering
from pprint import pprint

---
## 2. 전처리

### 2.1 데이터 불러오기

In [29]:
## 전처리 과정
dt = pd.read_csv("/Users/ku/googleDrive/laboratory/개인 논문/nudge_sentence.txt", delimiter="\t")

### 2.1 'Sentence' 열만 따로 리스트에 저장하기.

In [30]:
documents = list(dt['Sentence'])
print(documents[0:5])

['Using defaults in organ donation to increase compliance rates. Those countries where people are required to opt-out of organ donation report significantly higher consent than those with an opt-in policy. Possibly the most famous nudge, certainly the most eye-catching.', 'The authors sought to prime honesty by asking people to sign at the start of a form rather than the end when reporting how many miles they had driven on their car for insurance purposes. In this case there was a financial incentive to report less miles driven since reporting more would mean you would pay more (i.e. a higher number in the graph implies more honesty). The results indicated the treatment to be effective at inducing more honest declarations.', 'General Electric wanted its employees to stop smoking. They submitted to a Randomized Control Trial where the treatment group received cash incentives to quit. The control group received no incentives. Quitting for 6 months earned you $250, quitting for 12 months 

### 2.2 Tokenize(정규표현식을 이용하여 문자와 숫자만 추출)

In [31]:
sentences = [regexp_tokenize(document, '[\w]+') for document in documents]
texts = [[word for word in sentence if word not in stopwords.words('english') ] for sentence in sentences]
pprint(texts[1])

['The',
 'authors',
 'sought',
 'prime',
 'honesty',
 'asking',
 'people',
 'sign',
 'start',
 'form',
 'rather',
 'end',
 'reporting',
 'many',
 'miles',
 'driven',
 'car',
 'insurance',
 'purposes',
 'In',
 'case',
 'financial',
 'incentive',
 'report',
 'less',
 'miles',
 'driven',
 'since',
 'reporting',
 'would',
 'mean',
 'would',
 'pay',
 'e',
 'higher',
 'number',
 'graph',
 'implies',
 'honesty',
 'The',
 'results',
 'indicated',
 'treatment',
 'effective',
 'inducing',
 'honest',
 'declarations']


### 2.3(option) 모델 학습을 위해 클래스를 임의로 할당한다. (절반은 0, 나머지는 1)

In [32]:
# 모델이 제대로 돌아가는지 확인 하기 위해 데이터의 반을 클래스 0으로 나머지 반을 클래스 1로 할당한다.
dt.loc[:50, 'Class'] = 0
dt.loc[50:, 'Class'] = 1
print("\n클래스 '0': " + str(len(dt.loc[:50, 'Class'])) + "개")
print("클래스 '1': " + str(len(dt.loc[50:, 'Class'])) + "개")


클래스 '0': 51개
클래스 '1': 50개


### 2.4 클래스 0을 [0, 1], 클래스 1을 [1,0]으로 바꿔서 변수 y에 할당한다.

In [33]:
# 클래스를 0, 1 이 아니라 [0, 1], [0, 1]로 바꾸기.
y = []
for i in list(dt.loc[:, 'Class']):
    if i == 0:
        y.append([0, 1])
    else:
        y.append([1, 0])
y = np.asarray(y)
print("\nprint(y[0:10]): ")
print(y[0:10])


print(y[0:10]): 
[[0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]]


---
## 3. Embedding layer 생성

**VocabularyProcessor**  
: *'tf.nn.embedding_lookup'*의 ids로 활용하려면 단어에 '인덱스'가 할당 되어 있어야 하는데, 각 문서(문장)의 단어마다 **인덱스**를 할당해 주는 역할. 또한    문서마다 길이(단어의 개수)가 다를 수 있는데, *max_document_length*를 지정하여 문서의 길이를 같게 만들어 줌. (padding으로 0을 할당)


In [47]:
%%time 
# 셀 하나의 코드 수행시간을 출력 해 준다.
vocab_processor = tf.contrib.learn.preprocessing.VocabularyProcessor(max_document_length=200) # 객체 선언
word_index = np.array(list(vocab_processor.fit_transform(documents)))

# Extract word:id mapping from the object.
# word to ix 와 유사
vocab_dict = vocab_processor.vocabulary_._mapping

# Sort the vocabulary dictionary on the basis of values(id).
sorted_vocab = sorted(vocab_dict.items(), key=lambda x: x[1])

# Treat the id's as index into list and create a list of words in the ascending order of id's
# word with id i goes at index i of the list.
vocabulary = list(list(zip(*sorted_vocab))[0])
vocab_size = len(vocab_processor.vocabulary_)
print("vocab_size:", vocab_size)

vocab_size: 2391
CPU times: user 19.4 ms, sys: 965 µs, total: 20.4 ms
Wall time: 19.5 ms


In [48]:
# lookup table을 만들기 위해서 pre-trained word embedding을 불러온다. 
# word_embedding = gensim.models.KeyedVectors.load_word2vec_format("/Users/ku/Desktop/GoogleNews-vectors-negative300.bin", binary=True)

---
### 4. Foward propagation

In [70]:
'''
PARAMETERS:
    - sequence_length: sentence의 길이(= 문장의 단어 개수, token의 개수)
    - num_classes: class(label)의 개수
    - vocab_size: 단어 사전의 총 단어수
    - embedding_size: word vector의 차원
    - filter_sizes: 필터의 크기 e.g. "3,4,5"
    - num_filters: 필터의 개수, nonlinearity층의 차원 수와 관계 됨.
    - L2_reg_lambda: L2 정규화 파라미터.
    - filter_size: 필터의 크기 [3, 4, 5]
'''

sequence_length = word_index.shape[1]
num_classes = 2
batch_size = 16
embedding_size = 300
filter_size = [3, 4, 5]


""" 중복 코드로서 필요없지만... 혹시 모르니껜 남겨두자.
# embedding_layer를 만들기 위해 셋팅하자.
W = tf.Variable(tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0), name="W")
embedded_chars = tf.nn.embedding_lookup(W, input_x) # 차원수: [sequence length, embedding size]
embedded_chars_expanded = tf.expand_dims(embedded_chars, -1) 
# 차원수 : [sequence length * embedding size * 1]
"""

tf.reset_default_graph() # 기본 그래프를 초기화 한다.

# input과 관련 된 Placeholder를 설정한다.
orig_X = tf.placeholder(tf.int32, [None, sequence_length]) #tf.float32로 하면 tf.int32로 바꾸라며 에러가 난다.
orig_Y = tf.placeholder(tf.float32, [None, num_classes])
dropout_keep_prob = tf.placeholder(tf.float32)

# lookup table을 만든 후 위에서 만든 placeholder orig_X를 이용하여 임베딩 레이어를 생성한다.
lookup = tf.Variable(tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0, seed=123)) # Making look-up table
embedded_char = tf.nn.embedding_lookup(params=lookup, ids=orig_X)
# tensorflow conv2d 함수의 요구 차원을 맞추기 위해서...! 
X = tf.expand_dims(embedded_char, -1) # dimension: [None, sequence_length, embedding_size, 1]

# 필터를 만들어 준다.
W1_1 = tf.Variable(tf.truncated_normal(shape=[filter_size[0], embedding_size, 1, 2])) 
W1_2 = tf.Variable(tf.truncated_normal(shape=[filter_size[1], embedding_size, 1, 2]))
W1_3 = tf.Variable(tf.truncated_normal(shape=[filter_size[2], embedding_size, 1, 2]))
# dimension: [filter_size, embedding_size, inpurt_channel, output_channel]

b1_1 = tf.Variable(tf.constant(0.1), [2])
b1_2 = tf.Variable(tf.constant(0.1), [2])
b1_3 = tf.Variable(tf.constant(0.1), [2])
# dimension: [output_channel]

conv1_1 = tf.nn.relu(tf.nn.conv2d(X, W1_1, strides=[1, 1, 1, 1], padding="VALID") + b1_1)
conv1_2 = tf.nn.relu(tf.nn.conv2d(X, W1_2, strides=[1, 1, 1, 1], padding="VALID") + b1_2)
conv1_3 = tf.nn.relu(tf.nn.conv2d(X, W1_3, strides=[1, 1, 1, 1], padding="VALID") + b1_3)
# dimension: [None, sequence_length - filter_size + 1, 1, 2]

max1_1 = tf.nn.max_pool(conv1_1, ksize=[1, sequence_length - 3 + 1, 1, 1], strides=[1, 1, 1, 1], padding="VALID")
max1_2 = tf.nn.max_pool(conv1_2, ksize=[1, sequence_length - 4 + 1, 1, 1], strides=[1, 1, 1, 1], padding="VALID")
max1_3 = tf.nn.max_pool(conv1_3, ksize=[1, sequence_length - 5 + 1, 1, 1], strides=[1, 1, 1, 1], padding="VALID")
# dimension: [None, 1, 1, 2]

flatted_3 = tf.contrib.layers.flatten(max1_1)
flatted_4 = tf.contrib.layers.flatten(max1_2)
flatted_5 = tf.contrib.layers.flatten(max1_3)
# dimension: [None, 2]

dense = tf.concat([flatted_3, flatted_4, flatted_5], axis=1)
W_dense = tf.Variable(tf.truncated_normal([6, num_classes], stddev=0.1, seed=123), name="W")
b = tf.Variable(tf.constant(0.1), [num_classes])
Z = tf.matmul(dense, W_dense) + b
# dimesion: [None, 2]

print("\ninput: word_index[:3, :]" )
print("desired shape: [3, 2]")
print("result: ")
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(Z, feed_dict={orig_X: word_index[:3, :]}))


input: word_index[:3, :]
desired shape: [3, 2]
result: 
[[ 4.24278831  4.50464582]
 [ 6.16410971  3.63599467]
 [ 2.95482111  7.94880581]]


---
### 5. Loss, Back propagation, Weight update

In [37]:
%%time
# prediction = tf.cast(tf.argmax(Z, 1), tf.float32) # [None, 1]

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=Z, labels=orig_Y))

optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss)

# epoch: 30
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(30):
        _, cost = sess.run([optimizer, loss], feed_dict={orig_X: word_index, orig_Y: y})
        if i % 5 == 0:
            print("Epoch: " + str(i) + ", Cost: " + str(cost))

print("\nfinal cost:", cost, "\n")

Epoch: 0, Cost: 7.11484
Epoch: 5, Cost: 0.337379
Epoch: 10, Cost: 0.00141011
Epoch: 15, Cost: 0.0071311
Epoch: 20, Cost: 0.00172907
Epoch: 25, Cost: 0.000538102

final cost: 0.000314985 

CPU times: user 39.6 s, sys: 3.15 s, total: 42.8 s
Wall time: 13.5 s


In [38]:
print(cost)

0.000314985
